In [ ]:
%load_ext lab_black

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration
from azureml.core import Environment, Workspace

# Initialize Azure ML Workspace
ws = Workspace.from_config()

# Create a Python environment for the experiment
python_env = Environment.from_pip_requirements("pipeline_env", "requirements.txt")

In [ ]:
# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()
pipeline_run_config.environment = python_env

from azureml.core.compute import ComputeTarget

# Define the name of your existing compute target
existing_compute_target_name = "matthewhanauer3"

# Retrieve the existing compute target
compute_target = ComputeTarget(workspace=ws, name=existing_compute_target_name)

# Validate that the compute target exists
if compute_target is not None:
    print(f"Using existing compute target: {existing_compute_target_name}")
else:
    print(f"No compute target found with name {existing_compute_target_name}")
    exit(1)  # Exit the script if the compute target doesn't exist

# Modify the PythonScriptStep to include the compute_target
script_step = PythonScriptStep(
    name="Run Scheduled Script",
    script_name="data_modeling_ml.py",
    source_directory="./",
    runconfig=pipeline_run_config,
    compute_target=compute_target,
    allow_reuse=False,
)

In [ ]:
# Create the pipeline
pipeline = Pipeline(workspace=ws, steps=[script_step])


# Publish the pipeline
published_pipeline = pipeline.publish(
    name="My_Published_Pipeline",
    description="Published pipeline to run the script every minute.",
)

# Create an experiment and run the pipeline
experiment_name = "Scheduled_Script_Experiment"
experiment = Experiment(ws, experiment_name)

# Submit the pipeline as an experiment run
pipeline_run = experiment.submit(pipeline)

pipeline_id = published_pipeline.id

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Define the pipeline schedule
recurrence = ScheduleRecurrence(frequency="Minute", interval=1)  # Runs every minute
schedule = Schedule.create(
    ws,
    name="My_Schedule",
    description="Runs pipeline every minute",
    pipeline_id=pipeline_id,
    experiment_name=experiment_name,
    recurrence=recurrence,
)

In [ ]:
from azureml.core import Workspace
from azureml.pipeline.core import Schedule

# Initialize Azure ML Workspace
ws = Workspace.from_config()

# Your schedule name
schedule_name = "My_Schedule"

# List all schedules in the workspace
schedules = Schedule.list(ws)

# Filter and disable the schedule with the given name
for schedule in schedules:
    if schedule.name == schedule_name:
        print(f"Disabling schedule {schedule.name}")
        schedule.disable()